# Laboratorio 6
## Data Science

Autores:

- Nelson García 
- Christian Echeverría

## Análisis de Sentimientos de críticas de películas

Junto con Keras, viene un ejemplo imdb_lstm.py. Este ejercicio esta prácticamente basado en él.

Es un gran ejemplo del uso de las RNNs.  El conjunto de datos que se utilizará consta de críticas de películas generadas por usuarios, y una classificación indicando si le gustó, o no, basado en su rating asociado. 

Hay más información de este conjunto de datos en:

https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Como la comprensión del lenguaje escrito requiere "llevar cuenta" de todas las palabras en una oración, necesitamos una RNN para mantener una "memoria" de las palabras que pasaron antes, conforme va "leyendo" oraciones a lo largo del tiempo. 

En particular, se usarán unidades LSTM (Long Short-Term Memory) porque no es deseable "olvidar" palabras demasiado rápido...las palabras al inicio de una oración pueden afectar el significado de la misma grandemente.

Empezamos por la importación de lo que se requiere:

## Importación de Datos:

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"   # habilita growth desde el arranque
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"           

In [2]:
import tensorflow as tf

print("TF:", tf.__version__)
print("Build info:", getattr(tf.sysconfig, "get_build_info", lambda: {})())

gpus = tf.config.list_physical_devices('GPU')
print("GPUs detectadas:", gpus)

# Activa growth explícitamente (ya debería estar por env var, pero así confirmas)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers, regularizers


TF: 2.20.0
Build info: OrderedDict({'cpu_compiler': 'clang 18', 'cuda_compute_capabilities': ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90'], 'cuda_version': '12.5.1', 'cudnn_version': '9', 'is_cuda_build': True, 'is_rocm_build': False, 'is_tensorrt_build': False})
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Ahora importar los datos para entrenamiento y prueba.  Para que sea más manejable, se especifica que se quieren solamente las 50,000 palabras más populares en el conjunto de datos. Por algún motivo, este conjunto tiene una relación de 50% entreno y 50% prueba. 

In [3]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

Cargando los datos...


## Pre-procesamiento

In [4]:
len(X_train)

25000

In [5]:
len(X_test)

25000

A ver cómo son los datos, el primer elemento de entrenamiento debe ser una crítica de una película:

In [6]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

Esto no parece una crítica de una película!!!!  Resulta que la gente que preparó los datos ya hizo algo de preparación previa de los datos.  Estos números coinciden con el índice correspondiente a cada palabra de la crítica.  En realidad las palabras en sí, no son de interés...el modelo requiere números no palabras. 

Lo triste es que no será posible leer las críticas...siquiera para tener una idea de si funciona el análiisis, o no.

Y, ¿cómo son las etiquetas (metas)?

In [7]:
y_train[0]

np.int64(1)

Son simplemente 0 ó 1, que indica sí al que escribió la crítica le gustó, o no, la película.

Para resumir, para el entrenamiento se tiene un conjunto de críticas de películas que han sido convertidas a vectores de palabras representadas por enteros, y una clasificación de sentimiento binaria.

Las RNNs pueden "explotar" muy rápidamente (se habló de esto en clase).  Para que no se sobrecarguen las PCs que se podrían usar, se limitarán las críticas a las primeras 80 palabras:

In [8]:
X_train = sequence.pad_sequences(X_train, maxlen = 80)
X_test = sequence.pad_sequences(X_test, maxlen = 80)

### Extracción de features adicionales

In [10]:
import nltk

try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except LookupError:
    nltk.download('vader_lexicon', quiet=True)

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.sentiment import SentimentIntensityAnalyzer


# Inicializar el analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Función para obtener características adicionales
def extract_features(texts, max_length=80):
    features = []
    
    for text in texts:
        # Convertir el índice a palabras
        word_index = imdb.get_word_index()
        index_word = {v: k for k, v in word_index.items()}
        words = [index_word.get(i - 3, '') for i in text]  # Descontamos 3 por los valores reservados (pad, start, unknown)

        # 1. Longitud de la crítica
        features.append([len(words)])
        
        # 2. Proporción de palabras positivas/negativas
        pos_words, neg_words = 0, 0
        for word in words:
            if word:
                sentiment = sia.polarity_scores(word)
                if sentiment['compound'] > 0:
                    pos_words += 1
                elif sentiment['compound'] < 0:
                    neg_words += 1
        total_words = len([word for word in words if word])  # Palabras no vacías
        pos_neg_ratio = pos_words / neg_words if neg_words > 0 else pos_words
        
        features[-1].extend([pos_words, neg_words, pos_neg_ratio])
        
        # 3. Número de palabras únicas
        unique_words = len(set(words))
        features[-1].append(unique_words)
    
    return np.array(features)

# Extraer características de entrenamiento y prueba
train_features = extract_features(X_train)
test_features = extract_features(X_test)

# Concatenar características con las etiquetas de salida (y_train y y_test)
x_train_features = np.hstack((train_features, X_train))
x_test_features = np.hstack((test_features, X_test))

# Ahora x_train_features y x_test_features tienen las características adicionales


## Configuración del modelo



In [12]:
vocab_size = 50000
maxlen = int(X_train.shape[1])
n_features = int(train_features.shape[1])

### Modelo Simple

In [13]:
modelo = Sequential()
modelo.add(Embedding(50000, 128))
modelo.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
modelo.add(Dense(1, activation='sigmoid'))

I0000 00:00:1757570928.793712    5594 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1728 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:02:00.0, compute capability: 6.1


In [14]:
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Modelo con features

In [16]:
# Normalización de features adicionales dentro del grafo (evita fugas de datos)
normalizador = layers.Normalization(axis=-1, name="normalizador_features")
normalizador.adapt(train_features)

In [28]:
# ======================================================
# MODELO 1: LSTM básico + rama densa para features (simple)
# ======================================================
# Entrada 1: secuencia de palabras (tokens)
inp_seq_1 = layers.Input(shape=(maxlen,), name="input_secuencia")

# Rama de texto
x1 = layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True, name="embedding")(inp_seq_1)
x1 = layers.LSTM(64, name="lstm_64", use_cudnn=False)(x1)  # representación secuencial

# Entrada 2: features adicionales (numéricas)
inp_feat_1 = layers.Input(shape=(n_features,), name="input_features")
f1 = normalizador(inp_feat_1)
f1 = layers.Dense(32, activation="relu", name="feat_dense_32")(f1)

# Fusión de ramas
h1 = layers.Concatenate(name="concat")([x1, f1])
h1 = layers.Dense(64, activation="relu", name="post_concat_dense_64")(h1)
h1 = layers.Dropout(0.5, name="dropout_05")(h1)

# Capa de salida
out_1 = layers.Dense(1, activation="sigmoid", name="salida")(h1)

modelo_basico = keras.Model(inputs=[inp_seq_1, inp_feat_1], outputs=out_1, name="modelo_lstm_basico")

modelo_basico.compile(
    optimizer=optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC(name="auc"), "accuracy"]
)


### Modelo mejorado

In [54]:
def to_right_padding(arr, pad_value=0):
    arr = np.asarray(arr)
    out = np.full_like(arr, pad_value)
    for i in range(arr.shape[0]):
        row = arr[i]
        nz = row[row != pad_value]  # elimina ceros de pad
        L = nz.shape[0]
        out[i, :L] = nz             # tokens al inicio, pad al final
    return out

X_train2, X_test2 = X_train, X_test

X_train_ = to_right_padding(X_train2, pad_value=0)
X_test_  = to_right_padding(X_test2,  pad_value=0)

maxlen = int(X_train_.shape[1])
n_features = int(train_features.shape[1])

In [61]:
# ======================================================
# MODELO 2: Arquitectura avanzada (BiLSTM apiladas + regularización)
# - SpatialDropout1D para robustez
# - BiLSTM con return_sequences + BiLSTM final
# - MLP más profundo en rama de features
# - Regularización L2 y Dropout
# ======================================================
inp_seq_2 = layers.Input(shape=(maxlen,), name="input_secuencia_avanzado")

# Rama de texto avanzada
x2 = layers.Embedding(input_dim=vocab_size, output_dim=256, mask_zero=True, name="embedding_avanzado")(inp_seq_2)
x2 = layers.SpatialDropout1D(0.2, name="spatial_dropout")(x2)
x2 = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2,implementation=2), name="bilstm_128_rs")(x2)
x2 = layers.Dropout(0.3, name="dropout_seq_03")(x2)
x2 = layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2,implementation=2), name="bilstm_64")(x2)
x2 = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(1e-4), name="text_dense_64")(x2)

# Rama de features avanzada
inp_feat_2 = layers.Input(shape=(n_features,), name="input_features_avanzado")
f2 = normalizador(inp_feat_2)
f2 = layers.Dense(64, activation="relu", name="feat_dense_64")(f2)
f2 = layers.Dropout(0.3, name="feat_dropout_03")(f2)
f2 = layers.Dense(32, activation="relu", name="feat_dense_32_b")(f2)

# Fusión
h2 = layers.Concatenate(name="concat_avanzado")([x2, f2])
h2 = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(1e-4), name="fusion_dense_128")(h2)
h2 = layers.Dropout(0.5, name="fusion_dropout_05")(h2)
h2 = layers.Dense(64, activation="relu", name="fusion_dense_64")(h2)

out_2 = layers.Dense(1, activation="sigmoid", name="salida_avanzada")(h2)

modelo_avanzado = keras.Model(inputs=[inp_seq_2, inp_feat_2], outputs=out_2, name="modelo_lstm_avanzado")

modelo_avanzado.compile(
    optimizer=optimizers.Adam(3e-4),
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC(name="auc"), "accuracy"]
)


## Entrenamiento y Evaluación

### Modelo Simple

In [19]:
# Agregar detención temprana para monitorear el entrenamiento
detencion_temprana = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 3,
    verbose = 1
)

In [20]:
historia = modelo.fit(
    X_train, 
    y_train,
    batch_size = 64,  # Puede ajustarse de acuerdo a la memoria de GPU disponible 
    epochs = 15,
    verbose = 1,     
    validation_data = (X_test, y_test),
    callbacks = [detencion_temprana]
)

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 119s 282ms/step - accuracy: 0.7831 - loss: 0.4604 - val_accuracy: 0.8339 - val_loss: 0.3799
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 124s 318ms/step - accuracy: 0.8868 - loss: 0.2841 - val_accuracy: 0.8238 - val_loss: 0.3923
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 128s 326ms/step - accuracy: 0.9288 - loss: 0.1855 - val_accuracy: 0.8293 - val_loss: 0.4398
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 129s 330ms/step - accuracy: 0.9534 - loss: 0.1303 - val_accuracy: 0.8242 - val_loss: 0.4736
Epoch 4: early stopping


In [21]:
# Después del entrenamiento se puede revisar el historial
print(historia.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


In [22]:
modelo.save("Analisis_Sentimiento.keras")

In [23]:
from tensorflow.keras.models import load_model
modelo = load_model("Analisis_Sentimiento.keras")

OK, ahora a evaluar la exactitud del modelo:

In [24]:
perdida, exactitud = modelo.evaluate(X_test, y_test,
                            batch_size = 64,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

391/391 - 16s - 42ms/step - accuracy: 0.8242 - loss: 0.4736
Pérdida de la Prueba: 0.473602294921875
Exactitud de la Prueba (Test accuracy): 0.8241999745368958


### Modelo con features

In [25]:
# Callbacks
early_stop = callbacks.EarlyStopping(
    monitor="val_loss",
    mode="max",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    mode="max",
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

In [30]:
history_basico = modelo_basico.fit(
    [X_train, train_features],
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1,
)

Epoch 1/10
  4/313 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.4284 - auc: 0.3311 - loss: 0.7474 

I0000 00:00:1757572907.767101    8772 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.7758 - auc: 0.8602 - loss: 0.4659 - val_accuracy: 0.8382 - val_auc: 0.9212 - val_loss: 0.3598 - learning_rate: 0.0010
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9046 - auc: 0.9634 - loss: 0.2432 - val_accuracy: 0.8270 - val_auc: 0.9199 - val_loss: 0.4340 - learning_rate: 0.0010
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9542 - auc: 0.9876 - loss: 0.1348 - val_accuracy: 0.8272 - val_auc: 0.9069 - val_loss: 0.4741 - learning_rate: 0.0010
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9750 - auc: 0.9957 - loss: 0.0744 - val_accuracy: 0.8196 - val_auc: 0.8955 - val_loss: 0.5840 - learning_rate: 0.0010
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9858 - auc: 0.9983 - loss: 0.0422 - val_accuracy: 0.8210 - val_auc: 0.8844 - val_loss: 0.8399 - learning_rate: 0.0010
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9901 - auc: 0.9988

In [39]:
modelo_basico.save("Analisis_Sentimiento2.keras")

In [48]:
perdida, AUC, exactitud = modelo_basico.evaluate([X_test, test_features], y_test,
                            batch_size = 64,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)
print('AUC:', AUC)

391/391 - 2s - 4ms/step - accuracy: 0.8010 - auc: 0.8481 - loss: 1.3265
Pérdida de la Prueba: 1.3264585733413696
Exactitud de la Prueba (Test accuracy): 0.80103999376297
AUC: 0.8481361269950867


### Modelo mejorado

In [62]:
history_avanzado = modelo_avanzado.fit(
    [X_train_, train_features],
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 275s 835ms/step - accuracy: 0.7298 - auc: 0.8142 - loss: 0.5414 - val_accuracy: 0.8284 - val_auc: 0.9153 - val_loss: 0.3870 - learning_rate: 3.0000e-04
Epoch 2/10


2025-09-11 01:29:55.062262: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51200000 exceeds 10% of free system memory.


313/313 ━━━━━━━━━━━━━━━━━━━━ 274s 876ms/step - accuracy: 0.8816 - auc: 0.9465 - loss: 0.3099 - val_accuracy: 0.8380 - val_auc: 0.9240 - val_loss: 0.3673 - learning_rate: 3.0000e-04
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 291s 930ms/step - accuracy: 0.9421 - auc: 0.9814 - loss: 0.1817 - val_accuracy: 0.8460 - val_auc: 0.9248 - val_loss: 0.4053 - learning_rate: 1.5000e-04
Epoch 3: early stopping
Restoring model weights from the end of the best epoch: 1.


2025-09-11 01:39:20.895704: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51200000 exceeds 10% of free system memory.


In [63]:
modelo_avanzado.save("Analisis_Sentimiento3.keras")

2025-09-11 01:39:44.393003: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51200000 exceeds 10% of free system memory.
2025-09-11 01:39:44.638717: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51200000 exceeds 10% of free system memory.
2025-09-11 01:39:44.714026: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51200000 exceeds 10% of free system memory.


In [64]:
perdida, AUC, exactitud = modelo_avanzado.evaluate([X_test_, test_features], y_test,
                            batch_size = 64,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)
print('AUC:', AUC)

391/391 - 45s - 116ms/step - accuracy: 0.8362 - auc: 0.9156 - loss: 0.3859
Pérdida de la Prueba: 0.3858749270439148
Exactitud de la Prueba (Test accuracy): 0.8361600041389465
AUC: 0.9155688285827637


## Informe